In [ ]:
# default_exp utilities

In [ ]:
#export
import os
from skimage.measure import compare_ssim as SSIM #TODO: deprecated, fix that
import math
import numpy as np

In [ ]:
#export
# https://stackoverflow.com/a/62078664 
class Bunch(object):
    def __init__(self, adict):
        """Create a namespace object from a dict, recursively"""
        self.__dict__.update({k: self.__elt(v) for k, v in adict.items()})

    def __elt(self, elt):
        """Recurse into elt to create leaf namepace objects"""
        if type(elt) is dict:
            return type(self)(elt)
        if type(elt) in (list, tuple):
            return [self.__elt(i) for i in elt]
        return elt

    def __repr__(self):
        """Return repr(self)."""
        return "%s(%s)" % (type(self).__name__, repr(self.__dict__))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

In [ ]:
#export
def sanity_check(args):
    multiscale_architectures = ['MSResNet']
    if args.model.generator.architecture in multiscale_architectures:
        if args.model.loss.adversarial_loss.name:
            assert args.model.loss.adversarial_loss.multiscale
        if args.model.loss.content_loss.name:
            assert args.model.loss.content_loss.multiscale
        if args.model.loss.perceptual_loss.name:
            assert args.model.loss.perceptual_loss.multiscale
        assert args.input.pyramid_levels > 1
    else:
        if args.model.loss.adversarial_loss.name:
            assert args.model.loss.adversarial_loss.multiscale is None
        if args.model.loss.content_loss.name:
            assert args.model.loss.content_loss.multiscale is None
        if args.model.loss.perceptual_loss.name:
            assert args.model.loss.perceptual_loss.multiscale is None
        assert args.input.pyramid_levels == 1

In [ ]:
#export
# TODO: current version expects <input_image_path target_image_path> lines in the image_pair_list
# but it shouldn't have to, the reader should be able to deal with lines having only <target_image_path>
def get_GOPRO_lists(root_folder, image_pair_list):
    """Reads input image lists for the GOPRO dataset.
    
    This is based on my current folder structure in hekate so
    write your own function. the output should consist of a list
    with elements containing the FULL path (i.e. from the root).
    """
    input_list = []
    target_list = []
    with open(image_pair_list, 'r') as f:
        for line in f:
            input_filename = line.split(' ')[0].strip('\n').strip('\t')
            target_filename = line.split(' ')[1].strip('\n').strip('\t')
            input_filepath = os.path.join(root_folder, input_filename)
            target_filepath = os.path.join(root_folder, target_filename)
            input_list.append(input_filepath)
            target_list.append(target_filepath)
    return input_list, target_list

In [ ]:
#export
# TODO: ugly and opaque, rewrite these
def tensor2im(image_tensor, imtype=np.uint8):
    image_list = []
    for image in image_tensor:
        image_numpy = image.cpu().float().numpy()
        image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
        image_list.append(image_numpy)
    return np.asarray(image_list).astype(imtype)

# TODO: rewrite this, wtf are those variable names!
def get_metrics(input_, output, target) -> (float, float, np.ndarray):
    if isinstance(target, list):
        input_ = input_[-1]
        output = output[-1]
        target = target[-1]
    fake = tensor2im(output.data)
    real = tensor2im(target.data)
    psnrs = []
    ssims = []
    for fake_, real_ in zip(fake, real):
        psnr = PSNR(fake_, real_)
        ssim = SSIM(fake_, real_, multichannel=True)
        psnrs.append(psnr)
        ssims.append(ssim)
    psnr = np.mean(psnrs)
    ssim = np.mean(ssims)

    return psnr, ssim

In [ ]:
#export
# TODO: together with the get_metrics(), these need to be rewritten
# TODO: (maybe) put this in some metrics.py
# actually, rewrite in pytorch so that we can autograd if we want
def PSNR(img1, img2):
    mse = np.mean((img1 / 255. - img2 / 255.) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_utilities.ipynb.
Converted 02_architecture_common.ipynb.
Converted 03_architecture_MSResNet.ipynb.
Converted 04_architecture_DeblurGANv2.ipynb.
Converted 05_blurring.ipynb.
Converted 06_dataset.ipynb.
Converted 07_losses.ipynb.
Converted 08_callbacks.ipynb.
Converted 09_model.ipynb.
Converted fuckit.ipynb.
